In [1]:
!pip install -U transformers
!pip install -U huggingface_hub
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.6
    Uninstalling huggingface-hub-0.24.6:
      Successfully uninstalled huggingface-hub-0.24.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [2]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import os

In [3]:
train_url = 'https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/train.csv'
df = pd.read_csv(train_url)

In [4]:
df.head()

,question,answer
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor..."
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...


In [5]:
df.shape

(1985, 2)

In [6]:
dev_url = 'https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/dev.csv'
df_val = pd.read_csv(dev_url)

In [7]:
df_val.head()

,question,answer
0,How do U-Nets skip connections contribute to b...,Skip connections in U-Net preserve fine-graine...
1,Explain LSTM and its significance.,LSTM (Long Short-Term Memory) is a type of RNN...
2,How are N-Grams used?,N-Grams can be used to capture context and rel...
3,What is a token in NLP?,"A unit of text, such as a word or punctuation ..."
4,Does BoW give an ordered document representation?,No.


In [8]:
df_val.shape

(248, 2)

In [9]:
test_url = 'https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/test.csv'
df_test = pd.read_csv(test_url)

In [10]:
df_test.head()

,question,answer
0,What is feature selection?,The process of selecting a subset of relevant ...
1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...
2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch
3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...
4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...


In [11]:
df_test.shape

(249, 2)

In [12]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import set_seed
import evaluate

In [13]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [14]:
torch.manual_seed(42)
set_seed(42)

In [15]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>', sep_token='<|sep|>')
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50260, 768)

In [16]:
datacollator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm=False)

In [17]:
class QADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
            encodings_dict = tokenizer('<|startoftext|>' + row["question"] + '<|sep|>' + row["answer"] + '<|endoftext|>', truncation=True, max_length=300, padding="max_length", return_tensors='pt')
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

In [18]:
class ValQADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
          encodings_dict = tokenizer('<|startoftext|>' + row["question"] + '<|sep|>', truncation=True, max_length=120, padding="max_length", return_tensors='pt')
          if((encodings_dict['input_ids'][0][119] != torch.Tensor(np.array([50259]))) and (encodings_dict['input_ids'][0][119] != torch.Tensor(np.array([50258])))):
            encodings_dict['input_ids'][0] = torch.cat((encodings_dict['input_ids'][0][np.r_[:119]],torch.Tensor(np.array([50258]))),0)
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

In [19]:
class TestQADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
          encodings_dict = tokenizer('<|startoftext|>' + row["question"] + '<|sep|>', truncation=True, max_length=120, return_tensors='pt')
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

In [20]:
train_dataset = QADataset(df, tokenizer)
val_dataset = ValQADataset(df_val,tokenizer)

<ipython-input-17-4833a31b834d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
<ipython-input-17-4833a31b834d>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
<ipython-input-18-c06aa83ace77>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
<ipython-input-18-c06aa83ace77>:11: UserWarning: To copy construct from a tensor, it is recommended

In [21]:
# Define compute_metrics function
def compute_metrics(eval_pred, eval_dataset, df):
    decoded_preds = []
    references = df['answer']
    refs = []

    # Generate predictions for the validation dataset
    for i, sample_input in enumerate(eval_dataset):
        temp_input = sample_input[0][sample_input[0] != torch.Tensor(np.array([50259]))]
        temp_input = temp_input[None, :]
        metric_outputs = model.generate(temp_input.cuda(), min_new_tokens=60, max_new_tokens=200, penalty_alpha=0.6, top_k=4, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
        decoded_preds.append(tokenizer.decode(metric_outputs[0]))

    final_preds = []
    for j in range(len(decoded_preds)):
        lst = decoded_preds[j].split('<|sep|>')
        if len(lst) >= 2:
            temp_decoded = lst[1].replace("<|endoftext|>", "")
            final_preds.append(temp_decoded.split('___')[0])
        else:
            print(f'{j}: {lst[0]}')

        refs.append(references[j])

    # Compute BLEU, ROUGE, and METEOR metrics
    results_bleu = bleu.compute(predictions=final_preds, references=refs)
    results_rouge = rouge.compute(predictions=final_preds, references=refs)
    results_meteor = meteor.compute(predictions=final_preds, references=refs)

    # Return the computed metrics
    return {
        'bleu': results_bleu['bleu'],
        'rouge1': results_rouge['rouge1'],
        'rouge2': results_rouge['rouge2'],
        'rougeL': results_rouge['rougeL'],
        'meteor': results_meteor['meteor']
    }

In [22]:
# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    metric_for_best_model="rougeL",  # Change this to "rougeL"
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    report_to='none',
    save_total_limit=1
)

In [23]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids, labels

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=datacollator,
    compute_metrics = lambda pred: compute_metrics(pred, val_dataset, df_val),
    preprocess_logits_for_metrics = preprocess_logits_for_metrics)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [25]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
0,18.197400,20.135967,0.000410,0.111922,0.025360,0.096927,0.047108
1,3.629800,3.391601,0.023298,0.185109,0.050899,0.164044,0.161799
2,3.044100,2.843122,0.025836,0.196151,0.060708,0.173651,0.200782
4,2.592000,2.407003,0.034193,0.221523,0.072317,0.188336,0.224581
5,2.421100,2.291131,0.032924,0.214988,0.070746,0.185544,0.233820
6,2.334200,2.200982,0.035875,0.218208,0.074118,0.188757,0.240091
8,2.036100,2.090996,0.037341,0.221421,0.076443,0.191619,0.253458
9,1.955600,2.059655,0.037434,0.220612,0.073283,0.183678,0.250752
10,1.862100,2.047590,0.038152,0.225314,0.076143,0.189162,0.259146
12,1.701000,2.031643,0.041987,0.232655,0.080978,0.192810,0.264465


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # typ

TrainOutput(global_step=1240, training_loss=4.876816641899847, metrics={'train_runtime': 8509.6402, 'train_samples_per_second': 4.665, 'train_steps_per_second': 0.146, 'total_flos': 6054677337600000.0, 'train_loss': 4.876816641899847, 'epoch': 19.919678714859437})

In [26]:
trainer.save_model()

In [27]:
test_dataset = TestQADataset(df_test,tokenizer)

<ipython-input-19-6db5a894922d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
<ipython-input-19-6db5a894922d>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))


In [28]:
temp_output = []
for i, sample_input in enumerate(test_dataset):
    if(len(sample_input[0]) == 119 and sample_input[0][119] != torch.Tensor(np.array([50258]))):
        sample_input[0] = torch.cat((sample_input[0][np.r_[:119]],torch.Tensor(np.array([50258]))),0)
    #sample_output = model.generate(sample_input.cuda(), min_new_tokens = 60, max_new_tokens = 200, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    sample_output = model.generate(sample_input.cuda(), min_new_tokens = 60, max_new_tokens = 200, penalty_alpha=0.6, top_k=4, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    temp_output.append(tokenizer.decode(sample_output[0]))
final_output =[]
for j in range(len(temp_output)):
    lst = temp_output[j].split('<|sep|>')
    if (len(lst) >= 2):
        temp_decoded = lst[1].replace("<|endoftext|>","")
        final_output.append(temp_decoded.split('___')[0])
    else:
        final_output.append("")

In [29]:
df_test["Generated"] = final_output
df_test.to_csv('qna_gpt2_small_dataset_2_v1.csv')

In [30]:
pred = df_test['Generated']
ref = df_test['answer']

In [31]:
# Update score_evaluate function to use BLEU, ROUGE, and METEOR
def score_evaluate(predictions, references):
    preds = []
    refs = []

    # Prepare the predictions and references
    for i in range(len(predictions)):
        #print("i:", i)
        preds.append(predictions[i])
        refs.append(references[i])

    # Calculate BLEU, ROUGE, and METEOR scores
    results_bleu = bleu.compute(predictions=preds, references=refs)
    results_rouge = rouge.compute(predictions=preds, references=refs)
    results_meteor = meteor.compute(predictions=preds, references=refs)

    # Print evaluation metrics
    print("BLEU Score : " + str(results_bleu['bleu']))
    print("ROUGE-1 Score : " + str(results_rouge['rouge1']))
    print("ROUGE-2 Score : " + str(results_rouge['rouge2']))
    print("ROUGE-L Score : " + str(results_rouge['rougeL']))
    print("METEOR Score : " + str(results_meteor['meteor']))

In [32]:
# Evaluate the model predictions
score_evaluate(pred, ref)

BLEU Score : 0.047399287564197874
ROUGE-1 Score : 0.2403687228320825
ROUGE-2 Score : 0.08903295582756396
ROUGE-L Score : 0.19976541520824026
METEOR Score : 0.2801703539207304


In [33]:
df_generated = pd.read_csv("/content/qna_gpt2_small_dataset_2_v1.csv")

In [34]:
df_generated.head(10)

,Unnamed: 0,question,answer,Generated
0,0,What is feature selection?,The process of selecting a subset of relevant ...,Feature selection is the process of selecting ...
1,1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...,MDP involves predicting the next action in a s...
2,2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch,Batch normalization normalizes the inputs of a...
3,3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...,Word2Vec creates word embeddings by taking a v...
4,4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...,Hidden layers in MLP allow the model to learn ...
5,5,What is a Decision Tree?,A Decision Tree is a predictive learning algor...,A Decision Tree is a supervised learning algor...
6,6,What is variance in machine learning?,Variance refers to the error introduced by the...,Variance measures the difference between predi...
7,7,What are common methods to diagnose bias and v...,Common methods include analyzing learning curv...,Common methods include regression summary stat...
8,8,What is the difference between L1 and L2 regul...,L1 regularization adds the absolute values of ...,"L1 regularization penalizes the loss function,..."
9,9,What is feature engineering for unstructured d...,Feature engineering for unstructured data invo...,Feature engineering for unstructured data invo...


In [35]:
model.save_pretrained("qna_gpt2_small_dataset_2_v1")
tokenizer.save_pretrained("qna_gpt2_small_dataset_2_v1")

('qna_gpt2_small_dataset_2_v1/tokenizer_config.json',
 'qna_gpt2_small_dataset_2_v1/special_tokens_map.json',
 'qna_gpt2_small_dataset_2_v1/vocab.json',
 'qna_gpt2_small_dataset_2_v1/merges.txt',
 'qna_gpt2_small_dataset_2_v1/added_tokens.json')

In [36]:
from huggingface_hub import notebook_login

In [37]:
notebook_login()

In [39]:
repo_name="qna_gpt2_small_dataset_2_v1"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kkasiviswanath/qna_gpt2_small_dataset_2_v1/commit/8f02c53e9a8f2dfb1808697478868c19d1d3b82e', commit_message='Upload tokenizer', commit_description='', oid='8f02c53e9a8f2dfb1808697478868c19d1d3b82e', pr_url=None, pr_revision=None, pr_num=None)

In [40]:
# Define the cleaning function
def clean_answer(response):
    lst = response.split('<|sep|>')
    if len(lst) >= 2:
        response = lst[1].replace("<|endoftext|>", "").replace("<|pad|>", "").replace("<|startoftext|>", "")
        response = response.split('___')[0]
    return response

# Define the function to generate an answer based on a question
def generate_answer(question: str):
    prompt = f"<|startoftext|> {question} <|sep|>"

    # Generate the answer using the model
    sample_outputs = model.generate(
        tokenizer(prompt, return_tensors='pt')['input_ids'].cuda(),  # Moving input to GPU if necessary
        min_new_tokens=60,
        max_new_tokens=200,
        penalty_alpha=0.6,
        top_k=4,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Clean and decode the answer
    answer = clean_answer(tokenizer.decode(sample_outputs[0]))
    return answer

In [41]:
# Test the generate_answer function with a sample question
sample_question = "Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?"
generated_answer = generate_answer(sample_question)

# Print the generated answer
print("Question: ", sample_question)
print("Generated Answer: ", generated_answer)

Question:  Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?
Generated Answer:  Yes, we can. An autoencoder can reconstruct the original data from the compressed compressed form using techniques like dropout, which randomly drop units during training. This reduces dimensionality and helps in reducing the number of hidden layers. However, it is expensive and time-consuming to learn. In contrast, deep learning algorithms like GPT-3 can learn from this data.


In [42]:
from transformers import pipeline

# Load the pipeline with the custom GPT-2 model
qa_pipeline = pipeline('text-generation', model='kkasiviswanath/qna_gpt2_small_dataset_2_v1')

# Define a function to generate answers using the pipeline
def generate_answer_with_pipeline(question: str):
    prompt = f"<|startoftext|> {question} <|sep|>"
    output = qa_pipeline(prompt, max_new_tokens=200, top_k=4, pad_token_id=50256, return_full_text=False)
    answer = clean_answer(output[0]['generated_text'])
    return answer


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/71.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [46]:
# Example usage
question = "Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?"
answer = generate_answer_with_pipeline(question)
print("Generated Answer:", answer)

Generated Answer: Yes, we can. An autoencoder can decode input data into a lower-dimensional compressed representation, enabling the network to perform dimensionality reduction tasks even when the output data is not dimensionally separable.
